# Spendee Data

In [1]:
import pandas as pd
import altair as alt

PATH = "../data/transactions_export_2025-07-24_unfcu.csv"

df = pd.read_csv(PATH)

# Explore categories, totals, and time ranges
df

,Date,Wallet,Type,Category name,Amount,Currency,Note,Labels,Author
0,2021-11-11T21:02:30+00:00,UNFCU 🌏,Expense,Shopping,-24.654016,USD,Kenko UV FILTER 67MM,photography,Eduardo Cifuentes
1,2023-12-22T03:03:38+00:00,UNFCU 🌏,Expense,Gifts,-50.000000,USD,Vino matrimonio julu montelig,NaN,Eduardo Cifuentes
2,2023-12-25T13:09:10+00:00,UNFCU 🌏,Expense,Restaurant,-13.119902,USD,NaN,NaN,Eduardo Cifuentes
3,2023-12-25T13:09:49+00:00,UNFCU 🌏,Expense,Shopping,-15.306552,USD,Almohada avipm cuello,NaN,Eduardo Cifuentes
4,2023-12-25T13:10:07+00:00,UNFCU 🌏,Expense,Coffee,-2.951978,USD,NaN,NaN,Eduardo Cifuentes
...,...,...,...,...,...,...,...,...,...
2051,2025-07-07T13:40:16+00:00,UNFCU 🌏,Expense,Restaurant,-11.976334,USD,NaN,NaN,Eduardo Cifuentes
2052,2025-07-07T20:36:54+00:00,UNFCU 🌏,Expense,Healthcare,-94.743076,USD,Vavuna qdenga 1 dosis,NaN,Eduardo Cifuentes
2053,2025-07-08T00:39:42+00:00,UNFCU 🌏,Expense,Acommodation,-17.519964,USD,NaN,NaN,Eduardo Cifuentes
2054,2025-07-08T14:44:39+00:00,UNFCU 🌏,Expense,Coffee,-3.011120,USD,NaN,NaN,Eduardo Cifuentes


In [2]:
df.describe()

,Amount
count,2056.000000
mean,-17.948601
std,120.913962
min,-3115.000000
25%,-9.761000
50%,-4.739202
75%,-2.074212
max,1342.137500


# Cleaning

In [3]:
# cleaning

# category
df.rename(columns = {'Category name':'category_name'}, inplace=True)
df.info()

df["category_name"] = df["category_name"].astype("category")

df['date_format'] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m") ## %Y-%m-%d"
df['amount'] = df['Amount'].abs()

# filter
df = df[(df['Type'] == 'Expense') & (df['category_name'] != 'Savings')]


df.groupby('date_format').amount.sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2056 entries, 0 to 2055
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           2056 non-null   object 
 1   Wallet         2056 non-null   object 
 2   Type           2056 non-null   object 
 3   category_name  2056 non-null   object 
 4   Amount         2056 non-null   float64
 5   Currency       2056 non-null   object 
 6   Note           872 non-null    object 
 7   Labels         620 non-null    object 
 8   Author         2056 non-null   object 
dtypes: float64(1), object(8)
memory usage: 144.7+ KB


date_format
2021-11      24.654016
2023-12     441.892980
2024-01    1887.149485
2024-02    3323.043322
2024-03    9001.908236
2024-04     982.971423
2024-05    2048.615520
2024-06    2665.325404
2024-07    1931.010922
2024-08    1649.655835
2024-09    2972.830352
2024-10    3555.568772
2024-11    7037.861406
2024-12    1755.824694
2025-01     173.309960
2025-02     263.740000
2025-03      30.000000
2025-04      96.332543
2025-05      40.000000
2025-06     116.000000
2025-07     137.240494
Name: amount, dtype: float64

# Confi - Number Format

Show currency in CLP or USD

In [4]:
# formatting

# display
pd.options.display.float_format = 'CLP ${:,.0f}'.format

In [5]:
df.dtypes


Date               object
Wallet             object
Type               object
category_name    category
Amount            float64
Currency           object
Note               object
Labels             object
Author             object
date_format        object
amount            float64
dtype: object

## Budgets

In [6]:
# read budgets
import json

with open('../utils/budgets.json') as json_file:
    budgets = json.load(json_file)

budgets

{'Rent': 'Gastos fijos',
 'Sport': 'Gastos fijos',
 'Personal Care': 'Gastos fijos',
 'Groceries': 'Gastos fijos',
 'Utilities': 'Gastos fijos',
 'Coffee-Snacks': 'Chao culpa',
 'Restaurant': 'Chao culpa',
 'Alcohol': 'Chao culpa',
 'Activities': 'Chao culpa',
 'Shopping': 'Chao culpa',
 'Travel': 'Viajes',
 'Investments': 'Inversion',
 'Savings': 'Ahorro'}

# Plots

In [7]:
source = df

tool = alt.TooltipValue = 'amount'
tooltip=['date', 'price', 'symbol', alt.Tooltip('amount:Q', format=':,.0f')]


alt.Chart(source).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y='date_format',
    x='amount',
    color='category_name:N',
    tooltip = ['Note', alt.Tooltip("mean(amount):Q", format="$,.0f")]
).properties(height=600, width=800)

alt.Chart(...)

In [8]:
# scatter

# remove outliers
# new_df = df.drop(df[df['amount'] > 1_000_000].index)
# new_df_ii = new_df.drop(new_df[new_df['category_name'] == 'Rent'].index)

# TODO: exluce Rent and shopping over 1 M clp

alt.Chart(source).mark_circle().encode(
    alt.Y('category_name', scale=alt.Scale(zero=False)),
    alt.X('amount', scale=alt.Scale(zero=False, padding=1)),
    size='amount',
    color= 'category_name',
    tooltip= 'Note'
).properties(height=1000, width=600)

alt.Chart(...)